In [1]:
import requests
from datetime import datetime, timedelta
import time
import json
import sqlite3
import os
from tqdm import tqdm
import pandas as pd
# import psycopg2

In [2]:
api_key = "RGAPI-edb938b7-46f3-4bf5-8685-7ee6b6bcfd00"
headers = {
    "X-Riot-Token": api_key
}

In [103]:
# 챌린저, 그랜드마스터, 마스터 구간 솔로 랭크 게임을 플레이한 유저 정보 추출 // summonerId, summonerName, tier
user_info = []
info_to_extract = ['summonerId','summonerName','tier']
tier_list = ['challenger','grandmaster','master']

for tier in tier_list:
    url = f"https://kr.api.riotgames.com/lol/league/v4/{tier}leagues/by-queue/RANKED_SOLO_5x5"
    response = requests.get(url, headers=headers)

    user_tier = response.json()['tier']
    documents = response.json()['entries']
    for document in documents:
        document['tier'] = user_tier
        user_info.append({key: value for key, value in document.items() if key in info_to_extract})


In [110]:
# Match Id 정보 수집을 위해 summonerName 활용 PUUID(유저 고유식별번호) 수집

not_found_summoner = []

# 사용할 수 있는 프로그레스 바 초기화
progress_bar = tqdm(total=len(user_info), desc='Processing Documents')

for document in user_info:
    summonerName = document['summonerName']
    url_1 = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summonerName}"
    response_1 = requests.get(url_1, headers=headers)
    
    if response_1.status_code == 200:
        document['puuid'] = response_1.json().get('puuid')
            
    elif response_1.status_code == 429:
        # Rate limit exceeded 에러인 경우, 주기마다 한 번씩 출력하고 rate limit을 기다림
        time.sleep(5)
        continue
    elif response_1.status_code == 404:
        # 404 에러인 경우, 에러 메시지를 출력하지 않고 진행
        not_found_summoner.append(summonerName)
        continue
    else:
        # 다른 상태 코드에 대한 처리
        print(f"Error {response_1.status_code} occurred for summoner: {summonerName}")
        continue
    
    # 처리된 document에 대한 프로그레스 바 업데이트
    progress_bar.update(1)

# 프로그레스 바 종료
progress_bar.close()

Processing Documents:  83%|████████▎ | 11532/13816 [3:58:50<47:18,  1.24s/it]   


In [157]:
# 누락된 데이터 검색 및 제거
missing_data = []
for i in user_info:
    if 'puuid' not in i.keys():
        missing_data.append(i['summonerName'])

excluded_names = list(set(not_found_summoner) | set(missing_data))

filtered_data = [info for info in user_info_1 if info['summonerName'] not in excluded_names]

In [167]:
# 중간저장
json_file_path = f"{os.getcwd()}/user_info.json"
with open(json_file_path, 'w') as json_file:
    json.dump(filtered_data, json_file, indent=4)

In [4]:
# 불러오기
json_file_path = f"{os.getcwd()}/user_info.json"
with open(json_file_path, 'r') as file:
    user_info = json.load(file)

    

In [195]:
# puuid 활용 matchId 수집
match_info = []
no_play_summoner = []

# 사용할 수 있는 프로그레스 바 초기화
progress_bar = tqdm(total=len(user_info), desc='Processing Documents')

for document in user_info:
    puuid = document['puuid']
    url_2 = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    response_2 = requests.get(url_2, headers=headers)
    
    if response_2.status_code == 200:
        match_info.append({'puuid': puuid, 'match': response_2.json()})
            
    elif response_2.status_code == 429:
        # Rate limit exceeded 에러인 경우, 주기마다 한 번씩 출력하고 rate limit을 기다림
        time.sleep(5)
        continue
    elif response_2.status_code == 404:
        # 404 에러인 경우, 에러 메시지를 출력하지 않고 진행
        no_play_summoner.append(puuid)
        continue
    else:
        # 다른 상태 코드에 대한 처리
        print(f"Error {response_2.status_code} occurred summonerName: {document['summonerName']}, puuid: {puuid}")
        continue
    
    # 처리된 document에 대한 프로그레스 바 업데이트
    progress_bar.update(1)

# 프로그레스 바 종료
progress_bar.close()

Processing Documents:  85%|████████▍ | 9799/11543 [3:18:19<35:17,  1.21s/it]   


In [198]:
# 중간저장
json_file_path = f"{os.getcwd()}/match_info.json"
with open(json_file_path, 'w') as json_file:
    json.dump(match_info, json_file, indent=4)

In [156]:
# 불러오기
json_file_path = f"{os.getcwd()}/match_info.json"
with open(json_file_path, 'r') as file:
    match_info = json.load(file)

In [21]:
# matchId 중복 제거 / 리스트화
match_list = []
for document in match_info:
    for match in document['match']:
        match_list.append(match)
match_list = list(set(match_list))

In [62]:
# Timeline 데이터 수집에 시간 소요가 커 12만개 데이터 중 4만개 수집
match_list_sm = match_list[:40000]
len(match_list_sm)

# Timeline 데이터 수집
timeline = []

# 사용할 수 있는 프로그레스 바 초기화
progress_bar = tqdm(total=len(match_list_sm), desc='Processing Documents')

for matchId in match_list_sm:
    url_3 = f"https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}/timeline"
    response_3 = requests.get(url_3, headers=headers)
    
    if response_3.status_code == 200:
        timeline.append(response_3.json())
            
    elif response_3.status_code == 429:
        # Rate limit exceeded 에러인 경우, 주기마다 한 번씩 출력하고 rate limit을 기다림
        time.sleep(5)
        continue
    elif response_3.status_code == 404:
        # 404 에러인 경우, 에러 메시지를 출력하지 않고 진행

        continue
    else:
        # 다른 상태 코드에 대한 처리
        print(f"Error {response_3.status_code} occurred matchId: {matchId}")
        continue
    
    # 처리된 document에 대한 프로그레스 바 업데이트
    progress_bar.update(1)

# 프로그레스 바 종료
progress_bar.close()

# 전체 데이터는 dictionary 구조 이며 'metadata'와 'info' 항목으로 구성된다.
# 'metadata' 항목에서 가져가야할 데이터는 'matchId' : str / 'participants(puuid) : list (1~5 / 6~10 두 팀으로 이뤄짐)
# test['metadata']['matchId'] / test['metadata']['participants']
# 'info'를 살펴봤을 때 각 데이터는 1분 동안 발생한 이벤트로 기록되어있음.
# 예시로 test 데이터는 36개의 이벤트로 약 36분 동안 게임이 진행되었음을 알 수 있다. # len(test['info']['frames'])

# 중간저장
json_file_path = f"{os.getcwd()}/timeline.json"
with open(json_file_path, 'w') as json_file:
    json.dump(timeline, json_file, indent=4)

# 불러오기
json_file_path = f"{os.getcwd()}/timeline.json"
with open(json_file_path, 'r') as file:
    timeline = json.load(file)

Processing Documents:  88%|████████▊ | 35313/40000 [12:27:08<1:39:09,  1.27s/it]   


In [323]:
# timeline 데이터 양(51GB), 분석을 고려했을 때 시간이 부족할 것이라고 판단하여, 주제 변경

# timeline 데이터 수집 중단으로 수집된 데이터 기준으로 추출
matchId_list = [i['metadata']['matchId'] for i in timeline]

# game 데이터 수집
game_info = []
selected_keys = ['gameId', 'participants', 'teams']

# game_info.append(response_4.json())

# 사용할 수 있는 프로그레스 바 초기화
progress_bar = tqdm(total=len(matchId_list), desc='Processing Documents')

for matchId in matchId_list:
    url_4 = f"https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}"
    response_4 = requests.get(url_4, headers=headers)
    
    if response_4.status_code == 200:
        data = response_4.json()
        game_info.append({key: data['info'][key] for key in selected_keys if key in data['info']})
            
    elif response_4.status_code == 429:
        # Rate limit exceeded 에러인 경우, 주기마다 한 번씩 출력하고 rate limit을 기다림
        time.sleep(5)
        continue
    elif response_4.status_code == 404:
        # 404 에러인 경우, 에러 메시지를 출력하지 않고 진행

        continue
    else:
        # 다른 상태 코드에 대한 처리
        print(f"Error {response_4.status_code} occurred matchId: {matchId}")
        continue
    
    # 처리된 document에 대한 프로그레스 바 업데이트
    progress_bar.update(1)

# 프로그레스 바 종료
progress_bar.close()

Processing Documents:  86%|████████▋ | 30475/35313 [10:35:42<1:40:55,  1.25s/it]   


In [324]:
# 중간저장
json_file_path = f"{os.getcwd()}/game_info.json"
with open(json_file_path, 'w') as json_file:
    json.dump(game_info, json_file, indent=4)

In [8]:
# 불러오기
json_file_path = f"{os.getcwd()}/game_info.json"
with open(json_file_path, 'r') as file:
    game_info = json.load(file)
    

In [22]:
# Game Platform 정보 붙여주기
for document in game_info:
    document['gameId'] = f"KR_{document['gameId']}"

In [137]:
# 데이터 학습 및 데이터베이스 업로드에 사용할 데이터 추출
participant_list = ['assists','deaths','champLevel','goldEarned']
game_result = []
for document in game_info:
    # partipant 항목 중 필요한 데이터 수집
    blue_team = [sum([participant[idx] for participant in document['participants'][:5]]) for idx in participant_list]
    red_team = [sum([participant[idx] for participant in document['participants'][5:10]]) for idx in participant_list]
    # team 항목 중 필요한 데이터 추가
    blue_team.extend([int(item) for sublist in document['teams'][0]['objectives'].values() for item in sublist.values()])
    red_team.extend([int(item) for sublist in document['teams'][1]['objectives'].values() for item in sublist.values()])
    # 블루, 레드팀 데이터 통합
    result = blue_team+red_team
    # gameId 추가
    result.append(document['gameId'])
    # 블루팀 기준으로 승,패 레이블 추가
    result.append(int(document['teams'][0]['win']))
    game_result.append(result)

In [130]:
# Column 이름 설정
col_name = ['Blue_Assists','Blue_Deaths','Blue_Total_Level','Blue_Total_gold','Blue_first_baron','Blue_baron_kills','Blue_first_blood','Blue_Kills','Blue_first_dragon','Blue_dragon_kills',
'Blue_first_inhibitor','Blue_inhibitor_destroyed','Blue_first_riftherald','Blue_riftherald_kills','Blue_first_tower','Blue_tower_destroyed',
'Red_Assists','Red_Deaths','Red_Total_Level','Red_Total_gold','Red_first_baron','Red_baron_kills','Red_first_blood','Red_Kills','Red_first_dragon','Red_dragon_kills',
'Red_first_inhibitor','Red_inhibitor_destroyed','Red_first_riftherald','Red_riftherald_kills','Red_first_tower','Red_tower_destroyed','GameId','Win']
# DataFrame
data = pd.DataFrame(game_result,columns=col_name)
data = data[['GameId','Blue_Kills','Blue_Assists','Blue_Deaths','Blue_Total_Level','Blue_Total_gold','Blue_first_blood','Blue_first_dragon','Blue_dragon_kills','Blue_first_riftherald','Blue_riftherald_kills',
'Blue_first_baron','Blue_baron_kills','Blue_first_tower','Blue_tower_destroyed','Blue_first_inhibitor','Blue_inhibitor_destroyed',
'Red_Kills','Red_Assists','Red_Deaths','Red_Total_Level','Red_Total_gold','Red_first_blood','Red_first_dragon','Red_dragon_kills','Red_first_riftherald','Red_riftherald_kills',
'Red_first_baron','Red_baron_kills','Red_first_tower','Red_tower_destroyed','Red_first_inhibitor','Red_inhibitor_destroyed','Win']]

In [163]:
# dictionary value 값으로 list 형식의 데이터는 삽입 불가능하여 matchid 쪼개기
match_result = []
for document in match_info:
    for match_id in document['match']:
        match_result.append({'puuid': document['puuid'], 'match': match_id})
match_info = match_result

In [169]:
user_info = pd.DataFrame(user_info)
match_info = pd.DataFrame(match_info)

# SQLite 데이터베이스에 연결
conn = sqlite3.connect('LOL.db')

# 데이터프레임을 SQLite 데이터베이스에 테이블로 저장
user_info.to_sql('user_info', conn, if_exists='replace', index=False)
match_info.to_sql('match_info', conn, if_exists='replace', index=False)
data.to_sql('game_info', conn, if_exists='replace', index=False)

# 연결 종료
conn.close()

In [168]:
# csv 파일 저장
user_info.to_csv(f"{os.getcwd()}/user_info.csv", index=False)
match_info.to_csv(f"{os.getcwd()}/match_info.csv", index=False)
data.to_csv(f"{os.getcwd()}/game_info.csv", index=False)

# 참고

In [156]:
# Riot Developer
https://developer.riotgames.com/apis#match-v5/GET_getMatch
# puuid 활용 match id 추출
https://cestmavie.tistory.com/64
# LOL Match 데이터 활용 승부 예측
https://star7sss.tistory.com/372
# Flask / upload image and display
https://stackoverflow.com/questions/63507504/python-flask-app-upload-image-and-display
# SQLite 코드 작성 참고
https://github.com/jjlee93/ds-sa-db-api/blob/main/src/Part_3.py

11543

In [115]:
# PostgreSQL
# ElephantSQL 용량 부족
host="drona.db.elephantsql.com"
user="eabonfbm"
password="i2ZqUVIzbp7v05NjjwJDXLTJDPnPlJsY"
database="eabonfbm"

connection = psycopg2.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cur = connection.cursor()

cur.execute("DROP TABLE IF EXISTS user_info;")
cur.execute("""
CREATE TABLE user_info (
    puuid VARCHAR(100),
    summoner_name VARCHAR(100),
    summoner_id VARCHAR,
    tier VARCHAR(20)
    
    
    )
""")
for row in user_info[:20]:
    cur.execute(f"INSERT INTO user_info (puuid, summoner_name, summoner_id, tier) VALUES ('{row['puuid']}','{row['summonerName']}','{row['summonerId']}','{row['tier']}')")

connection.commit()

0

In [145]:
# SQLite

DB_FILENAME = 'LOL.db'
DB_FILEPATH = os.path.join(os.getcwd(), DB_FILENAME)
conn = sqlite3.connect(DB_FILEPATH)

cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS user_info;")
cur.execute("""
CREATE TABLE user_info (
    puuid VARCHAR(100),
    summoner_name VARCHAR(100),
    summoner_id VARCHAR,
    tier VARCHAR(20)
    
    
    )
""")
for row in user_info:
    cur.execute(f"INSERT INTO user_info (puuid, summoner_name, summoner_id, tier) VALUES ('{row['puuid']}','{row['summonerName']}','{row['summonerId']}','{row['tier']}')")

conn.commit()